In [ ]:
# חיבור ה-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, applications
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models, regularizers
from tensorflow import keras
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# הגדרת נתיב לתמונות שלך בגוגל דרייב
base_dir = '/content/drive/MyDrive/data-2.0'

# חלוקת הנתונים ל-70% אימון ו-30% בדיקה
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)  # הגדרת חלוקה

# יצירת Data Generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),  # שינוי גודל התמונות ל-150x150
    batch_size=32,
    class_mode='categorical',
    subset='training'  # 70% נתוני אימון
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # 30% נתוני בדיקה
    shuffle=False
)

Found 243 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [ ]:
# יצירת מודל DNN
model = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # מספר הקלאסים
])

# קומפילציה של המודל
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



# Callbacks setup
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_saves-2.0/DNN/model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# Train the model
model.fit(train_generator, epochs=100,
          validation_data=validation_generator,
          batch_size=32,

callbacks=[early_stopping, model_checkpoint])

# הערכת המודל
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Found 243 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4857 - loss: 11.7590
Epoch 1: val_loss improved from inf to 13.85647, saving model to /content/drive/MyDrive/best_model_saves 2.0/DNN/model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 69s 8s/step - accuracy: 0.4852 - loss: 12.2587 - val_accuracy: 0.4854 - val_loss: 13.8565
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.5503 - loss: 16.7804
Epoch 2: val_loss improved from 13.85647 to 6.89587, saving model to /content/drive/MyDrive/best_model_saves 2.0/DNN/model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 328ms/step - accuracy: 0.5500 - loss: 16.6088 - val_accuracy: 0.4854 - val_loss: 6.8959
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - accuracy: 0.5099 - loss: 8.3688
Epoch 3: val_loss improved from 6.89587 to 2.48056, saving model to /content/drive/MyDrive/best_model_saves 2.0/DNN/model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 536ms/step - accuracy: 0.5099 - loss: 8.3523 - val_accuracy: 0.4854 - val_loss: 2.4806
Epoch 4/100
8/8 ━

In [ ]:
reloaded_model = keras.models.load_model('/content/drive/MyDrive/best_model_saves-2.0/CNN/model.keras')  # reload
reloaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,485,832 (55.26 MB)

 Trainable params: 4,828,610 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,657,222 (36.84 MB)

In [ ]:
# Install necessary libraries (if not installed)
!pip install tensorflow scikit-learn

# Ensure validation_generator is defined and has labels
y_true = validation_generator.classes  # True labels

# Get model predictions
y_pred_probs = reloaded_model.predict(validation_generator)  # Get probability scores

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)  # Use for multi-class
# y_pred = (y_pred_probs > 0.5).astype(int)  # Use for binary classification

# Compute Accuracy, Precision, Recall, and F1-score
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')  # 'macro' for multi-class
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Print results
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 45s 14s/step
Accuracy:  0.7670
Precision: 0.7732
Recall:    0.7691
F1-score:  0.7664
